## Nayyera , Duaa , Mahum
## 21i1651 , 21i1667 , 21i1662

# Reading data and Preprocessing


In [3]:
import zipfile
import os
import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# prompt: read a zip folder from drive and unzip it

import zipfile
import os

# Specify the path to your zip file in Google Drive
zip_file_path = "/content/drive/MyDrive/Colab Notebooks/Deep Project.zip" # Replace with the actual path

# Specify the directory to extract the contents
extract_dir = "/content/extracted_files" # Or any desired directory

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_dir):
  os.makedirs(extract_dir)

try:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
    print(f"Successfully unzipped {zip_file_path} to {extract_dir}")
except FileNotFoundError:
  print(f"Error: Zip file not found at {zip_file_path}")
except zipfile.BadZipFile:
  print(f"Error: Invalid zip file at {zip_file_path}")
except Exception as e:
  print(f"An unexpected error occurred: {e}")


Successfully unzipped /content/drive/MyDrive/Colab Notebooks/Deep Project.zip to /content/extracted_files


In [ ]:
import os
import cv2
import numpy as np

# Parameters
FRAME_SIZE = 64  # Resize frames to 64x64 for training
SELECTED_CLASSES = ['Biking', 'Diving', 'PullUps', 'Skiing', 'YoYo']  # Actions
def preprocess_and_save_videos(input_dir, output_dir, frame_size):
    """
    Preprocess videos by resizing frames to a specific size and saving them in the corresponding action folders.
    """
    for class_name in SELECTED_CLASSES:
        class_input_path = os.path.join(input_dir, class_name)
        class_output_path = os.path.join(output_dir, class_name)

        # Create output directory if it doesn't exist
        os.makedirs(class_output_path, exist_ok=True)

        for video_name in os.listdir(class_input_path):
            video_path = os.path.join(class_input_path, video_name)
            cap = cv2.VideoCapture(video_path)
            frames = []

            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                # Resize and convert frames
                frame = cv2.resize(frame, (frame_size, frame_size))
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Grayscale
                frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR)  # Convert back to 3-channel for saving
                frames.append(frame)

            cap.release()

            if frames:
                # Save processed video
                output_video_path = os.path.join(class_output_path, video_name)
                height, width = frames[0].shape[:2]
                fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                out = cv2.VideoWriter(output_video_path, fourcc, 30, (width, height))

                for frame in frames:
                    out.write(frame)
                out.release()
                print(f"Processed and saved: {output_video_path}")


In [ ]:
# Paths for input and output
input_dataset_path = r'c:\Users\hp\Desktop\NAYYERA\nbayyera haha\train'  # Update to your dataset path
output_dataset_path = r'processed_videos'

# Preprocess and save videos
preprocess_and_save_videos(input_dataset_path, output_dataset_path, FRAME_SIZE)

Processed and saved: processed_videos\Biking\v_Biking_g01_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g01_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c06.avi
Processed and saved: processed_videos\Biking\v_Biking_g02_c07.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c02.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c03.avi
Processed and saved: processed_videos\Biking\v_Biking_g03_c04.avi
Processed and saved: processed_videos\Biking\v_Biking_g04_c01.avi
Processed and saved: processed_videos\Biking\v_Biking_g04_c02.avi
Processed 

# SEQUENCE GENERATING

In [ ]:
import os
import numpy as np
import cv2

# Parameters
SEQUENCE_LENGTH = 10  # Input sequence length
PREDICT_FRAMES = 5  # Frames to predict

def load_video_frames(video_path, frame_size):
    """Load video frames from a file and preprocess."""
    cap = cv2.VideoCapture(video_path)
    frames = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (frame_size, frame_size))
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Grayscale
        frames.append(frame)

    cap.release()
    return np.array(frames)  # Shape: (num_frames, frame_size, frame_size)

In [ ]:
def save_sequences(sequence_folder, action, video_name, X, y):
    """Save the input-output sequences as .npy files."""
    action_folder = os.path.join(sequence_folder, action)
    os.makedirs(action_folder, exist_ok=True)  # Create action folder if not exists

    video_base = os.path.splitext(video_name)[0]
    X_file = os.path.join(action_folder, f"{video_base}_X.npy")
    y_file = os.path.join(action_folder, f"{video_base}_y.npy")

    np.save(X_file, X)
    np.save(y_file, y)

In [ ]:
def generate_sequences(dataset_path, sequence_folder, sequence_length, predict_frames, frame_size):
    """Generate input-output sequences for ConvLSTM training and save them."""
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if not os.path.isdir(class_path):
            continue  # Skip if not a directory

        for video_name in os.listdir(class_path):
            video_path = os.path.join(class_path, video_name)
            frames = load_video_frames(video_path, frame_size)

            # Ensure enough frames for at least one sequence
            required_frames = sequence_length + predict_frames
            if len(frames) < required_frames:
                print(f"Skipping {video_name} (Insufficient frames)")
                continue

            # Generate sequences
            num_sequences = len(frames) - required_frames + 1
            X, y = [], []
            for i in range(num_sequences):
                X.append(frames[i:i + sequence_length])  # Input sequence
                y.append(frames[i + sequence_length:i + sequence_length + predict_frames])  # Target sequence

            X = np.array(X)  # Shape: (num_samples, sequence_length, frame_size, frame_size)
            y = np.array(y)  # Shape: (num_samples, predict_frames, frame_size, frame_size)

            # Save sequences
            save_sequences(sequence_folder, class_name, video_name, X, y)


In [ ]:
# Paths
dataset_path = r'processed_videos'
sequence_folder = r'sequences'
frame_size = 64

# Generate and save sequences
generate_sequences(dataset_path, sequence_folder, SEQUENCE_LENGTH, PREDICT_FRAMES, frame_size)
print("Sequences generated and saved.")


Sequences generated and saved.


# Convolutional LSTM form scratch  

In [2]:
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import gc
from torch.utils.data import Dataset, DataLoader
from skimage.metrics import structural_similarity as compare_ssim

# Memory management utilities
def check_memory():
    """Check and print current memory usage."""
    print(f"Current Memory Usage: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

def clear_memory():
    """Clear PyTorch and Python memory."""
    gc.collect()
    torch.cuda.empty_cache()

# ConvLSTM Cell
class ConvLSTMCell(nn.Module):
    def __init__(self, input_channels, hidden_channels, kernel_size, bias=True):
        super(ConvLSTMCell, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.kernel_size = kernel_size
        self.padding = kernel_size // 2
        self.bias = bias

        self.conv = nn.Conv2d(
            in_channels=self.input_channels + self.hidden_channels,
            out_channels=4 * self.hidden_channels,
            kernel_size=self.kernel_size,
            padding=self.padding,
            bias=self.bias
        )

    def forward(self, x, h, c):
        combined = torch.cat([x, h], dim=1)
        gates = self.conv(combined)
        i, f, o, g = torch.split(gates, self.hidden_channels, dim=1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        o = torch.sigmoid(o)
        g = torch.tanh(g)

        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, height, width, device):
        h = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        c = torch.zeros(batch_size, self.hidden_channels, height, width, device=device)
        return h, c

# Video Frame Prediction Model
class VideoFramePredictionModel(nn.Module):
    def __init__(self, input_channels=1, hidden_channels=64, num_layers=2, kernel_size=3, predict_frames=5):
        super(VideoFramePredictionModel, self).__init__()
        self.input_channels = input_channels
        self.hidden_channels = hidden_channels
        self.num_layers = num_layers
        self.predict_frames = predict_frames

        self.encoder = nn.Conv2d(
            in_channels=input_channels,
            out_channels=hidden_channels,
            kernel_size=kernel_size,
            padding=kernel_size // 2
        )

        self.convlstm_layers = nn.ModuleList([
            ConvLSTMCell(
                input_channels=hidden_channels,
                hidden_channels=hidden_channels,
                kernel_size=kernel_size
            ) for _ in range(num_layers)
        ])

        self.decoder = nn.Conv2d(
            in_channels=hidden_channels,
            out_channels=input_channels,
            kernel_size=kernel_size,
            padding=kernel_size // 2
        )

    def forward(self, x):
        batch_size, seq_len, height, width = x.shape
        x = x.unsqueeze(1)

        device = x.device
        encoded_frames = [self.encoder(x[:, :, t, :, :]) for t in range(seq_len)]

        hidden_states = [layer.init_hidden(batch_size, height, width, device) for layer in self.convlstm_layers]

        for t in range(seq_len):
            frame = encoded_frames[t]
            for i, layer in enumerate(self.convlstm_layers):
                h, c = hidden_states[i]
                h, c = layer(frame, h, c)
                hidden_states[i] = (h, c)
                frame = h

        predicted_frames = []
        current_frame = encoded_frames[-1]

        for _ in range(self.predict_frames):
            for i, layer in enumerate(self.convlstm_layers):
                h, c = hidden_states[i]
                h, c = layer(current_frame, h, c)
                hidden_states[i] = (h, c)
                current_frame = h

            pred_frame = self.decoder(current_frame)
            predicted_frames.append(pred_frame.squeeze(1))

        return torch.stack(predicted_frames, dim=1)

# Dataset for Frame Prediction
class FramePredictionDataset(Dataset):
    def __init__(self, sequence_dir, sequence_length=10, predict_frames=5, max_sequences=None):
        self.inputs = []
        self.targets = []

        total_loaded_sequences = 0
        for class_dir in os.listdir(sequence_dir):
            class_path = os.path.join(sequence_dir, class_dir)
            for file in os.listdir(class_path):
                if file.endswith('_X.npy'):
                    input_path = os.path.join(class_path, file)
                    target_path = os.path.join(class_path, file.replace('_X.npy', '_y.npy'))

                    try:
                        input_sequences = np.load(input_path).astype(np.float32) / 255.0
                        target_sequences = np.load(target_path).astype(np.float32) / 255.0

                        self.inputs.extend(input_sequences)
                        self.targets.extend(target_sequences)

                        total_loaded_sequences += len(input_sequences)
                        if max_sequences and total_loaded_sequences >= max_sequences:
                            break
                    except Exception as e:
                        print(f"Error loading file {file}: {e}")
                        continue

        self.inputs = np.array(self.inputs)
        self.targets = np.array(self.targets)

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.inputs[idx], dtype=torch.float32),
            torch.tensor(self.targets[idx], dtype=torch.float32)
        )

# Training Function
def train_model(model, dataloader, num_epochs=10, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0.0
        start_time = time.time()

        for batch_idx, (inputs, targets) in enumerate(dataloader):
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            predictions = model(inputs)
            loss = criterion(predictions, targets)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Time: {time.time() - start_time:.2f}s")

    torch.save(model.state_dict(), 'convlstm_model.pth')
    print("Model saved.")

# Main Function
def main():
    sequence_dir = r'sequences'
    batch_size = 8
    num_epochs = 100
    max_sequences = 100  # Reduce dataset size for testing

    dataset = FramePredictionDataset(sequence_dir, max_sequences=max_sequences)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)

    model = VideoFramePredictionModel(input_channels=1, hidden_channels=64, num_layers=2, predict_frames=5)
    train_model(model, dataloader, num_epochs=num_epochs)

if __name__ == "__main__":
    main()


Epoch 1/100, Loss: 2.8796, Time: 29.54s
Epoch 2/100, Loss: 0.8358, Time: 29.84s
Epoch 3/100, Loss: 0.7404, Time: 31.12s
Epoch 4/100, Loss: 0.7095, Time: 29.87s
Epoch 5/100, Loss: 0.6769, Time: 29.36s
Epoch 6/100, Loss: 0.6635, Time: 29.26s
Epoch 7/100, Loss: 0.6379, Time: 29.21s
Epoch 8/100, Loss: 0.6164, Time: 29.19s
Epoch 9/100, Loss: 0.6025, Time: 29.17s
Epoch 10/100, Loss: 0.5953, Time: 29.09s
Epoch 11/100, Loss: 0.5753, Time: 29.28s
Epoch 12/100, Loss: 0.5598, Time: 29.45s
Epoch 13/100, Loss: 0.5455, Time: 29.84s
Epoch 14/100, Loss: 0.5428, Time: 30.54s
Epoch 15/100, Loss: 0.5315, Time: 29.73s
Epoch 16/100, Loss: 0.5346, Time: 29.86s
Epoch 17/100, Loss: 0.5237, Time: 30.39s
Epoch 18/100, Loss: 0.5268, Time: 31.34s
Epoch 19/100, Loss: 0.5059, Time: 30.03s
Epoch 20/100, Loss: 0.5081, Time: 29.79s
Epoch 21/100, Loss: 0.5002, Time: 29.73s
Epoch 22/100, Loss: 0.4923, Time: 29.75s
Epoch 23/100, Loss: 0.4968, Time: 29.62s
Epoch 24/100, Loss: 0.4946, Time: 29.63s
Epoch 25/100, Loss: 0.485

## predicting next frame

In [3]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import DataLoader

# Predict next frames
def predict_next_frames(model, dataloader, device, output_dir='predicted_frames', frame_size=(64, 64)):
    os.makedirs(output_dir, exist_ok=True)

    for batch_idx, (inputs, _) in enumerate(dataloader):
        inputs = inputs.to(device)
        predictions = model(inputs).detach().cpu().numpy()

        for seq_idx in range(predictions.shape[0]):
            seq_frames = predictions[seq_idx]

            # Save each predicted frame as an image
            for frame_idx, frame in enumerate(seq_frames):
                frame_path = os.path.join(output_dir, f"batch_{batch_idx}_seq_{seq_idx}_frame_{frame_idx}.png")
                frame_image = (frame * 255).astype(np.uint8).squeeze()  # Rescale and squeeze for grayscale
                frame_image = cv2.resize(frame_image, frame_size)  # Resize for consistency
                cv2.imwrite(frame_path, frame_image)
                print(f"Saved frame: {frame_path}")

# Create video from predicted frames
def make_video_from_frames(frame_dir, video_path='predicted_video.avi', frame_size=(64, 64), fps=5):
    frame_paths = sorted([
        os.path.join(frame_dir, f) for f in os.listdir(frame_dir) if f.endswith('.png')
    ])
    
    if not frame_paths:
        print("No frames found in the directory.")
        return

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(video_path, fourcc, fps, frame_size)

    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        video_writer.write(frame)
        print(f"Added frame to video: {frame_path}")

    video_writer.release()
    print(f"Video saved at {video_path}")

# Main prediction and video creation function
def predict_and_create_video(sequence_dir, model, device, frame_size=(64, 64), num_samples=5, predict_frames=5):
    # Load dataset
    dataset = FramePredictionDataset(sequence_dir, max_sequences=num_samples)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=0)

    # Predict next frames and save them as images
    output_dir = 'predicted_frames_convlstm'
    predict_next_frames(model, dataloader, device, output_dir, frame_size)

    # Create video from the saved frames
    video_path = 'predicted_video_convlstm.avi'
    make_video_from_frames(output_dir, video_path, frame_size)

# Example usage
if __name__ == "__main__":
    sequence_dir = r'sequences'
    model_path = 'convlstm_model.pth'

    # Load trained model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = VideoFramePredictionModel(input_channels=1, hidden_channels=64, num_layers=2, predict_frames=5)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    # Predict and create video
    predict_and_create_video(sequence_dir, model, device)


C:\Users\Me\AppData\Local\Temp\ipykernel_16404\33519252.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device

Saved frame: predicted_frames_convlstm\batch_0_seq_0_frame_0.png
Saved frame: predicted_frames_convlstm\batch_0_seq_0_frame_1.png
Saved frame: predicted_frames_convlstm\batch_0_seq_0_frame_2.png
Saved frame: predicted_frames_convlstm\batch_0_seq_0_frame_3.png
Saved frame: predicted_frames_convlstm\batch_0_seq_0_frame_4.png
Saved frame: predicted_frames_convlstm\batch_1_seq_0_frame_0.png
Saved frame: predicted_frames_convlstm\batch_1_seq_0_frame_1.png
Saved frame: predicted_frames_convlstm\batch_1_seq_0_frame_2.png
Saved frame: predicted_frames_convlstm\batch_1_seq_0_frame_3.png
Saved frame: predicted_frames_convlstm\batch_1_seq_0_frame_4.png
Saved frame: predicted_frames_convlstm\batch_2_seq_0_frame_0.png
Saved frame: predicted_frames_convlstm\batch_2_seq_0_frame_1.png
Saved frame: predicted_frames_convlstm\batch_2_seq_0_frame_2.png
Saved frame: predicted_frames_convlstm\batch_2_seq_0_frame_3.png
Saved frame: predicted_frames_convlstm\batch_2_seq_0_frame_4.png
Saved frame: predicted_fr

# Transformer-based model for frame generation with an emphasis on long-term dependencies.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define Dataset
class FrameDataset(Dataset):
    def __init__(self, num_samples=1000, seq_len=10, feature_dim=128):
        self.data = torch.randn(num_samples, seq_len, feature_dim)
        self.labels = torch.randn(num_samples, seq_len, feature_dim)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Define Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(TransformerEncoder, self).__init__()
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)

    def forward(self, src):
        return self.encoder(src)

# Define Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, d_model, nhead, num_layers):
        super(TransformerDecoder, self).__init__()
        self.decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead)
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_layers)
        self.output_proj = nn.Linear(d_model, d_model)

    def forward(self, tgt, memory):
        x = self.decoder(tgt, memory)
        return self.output_proj(x)

import torch
import torch.nn as nn

class TransformerFrameGenerator(nn.Module):
    def __init__(self, input_dim, output_dim, d_model, nhead, num_encoder_layers, num_decoder_layers, max_seq_len):
        super(TransformerFrameGenerator, self).__init__()
        
        self.d_model = d_model
        self.max_seq_len = max_seq_len
        
        self.embedding = nn.Linear(input_dim, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, max_seq_len, d_model))
        
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead), num_layers=num_encoder_layers)
        self.decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead), num_layers=num_decoder_layers)
        
        self.fc_out = nn.Linear(d_model, output_dim)
    
    def forward(self, src, tgt):
        # Add positional encoding
        src = self.embedding(src) + self.positional_encoding[:, :src.size(1), :]
        tgt = self.embedding(tgt) + self.positional_encoding[:, :tgt.size(1), :]
        
        # Transformer encoding
        memory = self.encoder(src)
        
        # Transformer decoding
        output = self.decoder(tgt, memory)
        
        # Final output
        out = self.fc_out(output)
        
        return out

# Define Training Logic
def train_model(model, dataloader, num_epochs=10, learning_rate=1e-4):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        total_loss = 0
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            tgt_inputs = torch.zeros_like(targets)
            tgt_inputs[:, 1:] = targets[:, :-1]  # Shift target inputs by one
            
            optimizer.zero_grad()
            predictions = model(inputs, tgt_inputs)
            loss = criterion(predictions, targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

# Save the model
def save_model(model, path="transformer_model.pth"):
    torch.save(model.state_dict(), path)
    print(f"Model saved to {path}")

# Main Function
def main():
    # Hyperparameters
    num_samples = 1000
    seq_len = 10
    feature_dim = 128
    d_model = 128
    nhead = 8
    num_encoder_layers = 4
    num_decoder_layers = 4
    batch_size = 16
    num_epochs = 100
    learning_rate = 1e-4

    # Dataset and DataLoader
    dataset = FrameDataset(num_samples=num_samples, seq_len=seq_len, feature_dim=feature_dim)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    
    # Updated model configuration
    model = TransformerFrameGenerator(
        input_dim=128,  # Feature dimension
        output_dim=128,  # Feature dimension of generated frames
        d_model=256,  # Increased d_model for better capacity
        nhead=8,
        num_encoder_layers=6,  # Increased depth
        num_decoder_layers=6,  # Increased depth
        max_seq_len=10  # Sequence length (number of frames)
    )
    model.to(device)

    # Train the model
    train_model(model, dataloader, num_epochs=num_epochs, learning_rate=learning_rate)

    # Save the trained model
    save_model(model)

# Device Configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if __name__ == "__main__":
    main()

C:\Users\Me\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/100, Loss: 1.0266
Epoch 2/100, Loss: 1.0100
Epoch 3/100, Loss: 1.0089
Epoch 4/100, Loss: 1.0082
Epoch 5/100, Loss: 1.0073
Epoch 6/100, Loss: 1.0070
Epoch 7/100, Loss: 1.0066
Epoch 8/100, Loss: 1.0060
Epoch 9/100, Loss: 1.0059
Epoch 10/100, Loss: 1.0051
Epoch 11/100, Loss: 1.0047
Epoch 12/100, Loss: 1.0044
Epoch 13/100, Loss: 1.0042
Epoch 14/100, Loss: 1.0034
Epoch 15/100, Loss: 1.0034
Epoch 16/100, Loss: 1.0029
Epoch 17/100, Loss: 1.0022
Epoch 18/100, Loss: 1.0007
Epoch 19/100, Loss: 0.9989
Epoch 20/100, Loss: 0.9974
Epoch 21/100, Loss: 0.9945
Epoch 22/100, Loss: 0.9917
Epoch 23/100, Loss: 0.9877
Epoch 24/100, Loss: 0.9832
Epoch 25/100, Loss: 0.9770
Epoch 26/100, Loss: 0.9694
Epoch 27/100, Loss: 0.9610
Epoch 28/100, Loss: 0.9505
Epoch 29/100, Loss: 0.9400
Epoch 30/100, Loss: 0.9276
Epoch 31/100, Loss: 0.9152
Epoch 32/100, Loss: 0.9037
Epoch 33/100, Loss: 0.8907
Epoch 34/100, Loss: 0.8787
Epoch 35/100, Loss: 0.8662
Epoch 36/100, Loss: 0.8540
Epoch 37/100, Loss: 0.8419
Epoch 38/1

# Predicting next frames 

In [2]:
import os
import cv2
import torch
import numpy as np
from torch.utils.data import DataLoader

# Define a dataset for frame prediction
class FramePredictionDataset(Dataset):
    def __init__(self, num_samples=1000, seq_len=10, feature_dim=128):
        self.data = torch.randn(num_samples, seq_len, feature_dim)
        self.labels = torch.randn(num_samples, seq_len, feature_dim)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]


# Create video from predicted frames
def make_video_from_frames(frame_dir, video_path, frame_size=(64, 64), fps=5):
    frame_paths = sorted([
        os.path.join(frame_dir, f) for f in os.listdir(frame_dir) if f.endswith('.png')
    ])

    if not frame_paths:
        print("No frames found in the directory.")
        return

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    video_writer = cv2.VideoWriter(video_path, fourcc, fps, frame_size)

    for frame_path in frame_paths:
        frame = cv2.imread(frame_path)
        video_writer.write(frame)
        print(f"Added frame to video: {frame_path}")

    video_writer.release()
    print(f"Video saved at {video_path}")

# Main prediction and video creation function
def predict_and_create_video(model, dataloader, device, frame_size=(64, 64), output_dir='predicted_frames_transformer'):
    # Predict next frames and save them as images
    predict_next_frames(model, dataloader, device, output_dir, frame_size)

def predict_next_frames(model, dataloader, device, output_dir='predicted_frames_transformer', frame_size=(64, 64)):
    os.makedirs(output_dir, exist_ok=True)

    for batch_idx, (inputs, _) in enumerate(dataloader):
        inputs = inputs.to(device)
        tgt_seq = torch.zeros_like(inputs).to(device)  # Initial target sequence
        
        predictions = []

        for step in range(inputs.shape[1]):  # Iterate over each time step
            pred = model(inputs[:, :step+1], tgt_seq[:, :step+1])  # Provide the input and target (shifted sequence)
            predictions.append(pred[:, step])  # Save the current step's prediction

            # Feed the predicted frame back into the target sequence for the next step
            if step + 1 < inputs.shape[1]:  # Ensure that we don't exceed the sequence length
                tgt_seq[:, step+1] = pred[:, step]  # Feed back the prediction for the next step

        predictions = torch.stack(predictions, dim=1).detach().cpu().numpy()

        # Save each predicted frame as an image
        for seq_idx in range(predictions.shape[0]):
            seq_frames = predictions[seq_idx]
            for frame_idx, frame in enumerate(seq_frames):
                frame_path = os.path.join(output_dir, f"batch_{batch_idx}_seq_{seq_idx}_frame_{frame_idx}.png")
                frame_image = (frame * 255).astype(np.uint8).squeeze()  # Rescale and squeeze for grayscale
                frame_image = cv2.resize(frame_image, frame_size)  # Resize for consistency
                cv2.imwrite(frame_path, frame_image)
                print(f"Saved frame: {frame_path}")


    # Create video from the saved frames
    video_path = 'predicted_video_transformer.avi'
    make_video_from_frames(output_dir, video_path, frame_size)

# Example usage
if __name__ == "__main__":
    # Hyperparameters
    num_samples = 1000
    seq_len = 10
    feature_dim = 128
    batch_size = 16
    frame_size = (64, 64)  # Define frame size for video

    # Dataset and DataLoader
    dataset = FramePredictionDataset(num_samples=num_samples, seq_len=seq_len, feature_dim=feature_dim)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Initialize Model
    model = TransformerFrameGenerator(
        input_dim=feature_dim, output_dim=feature_dim, d_model=128, 
        nhead=8, num_encoder_layers=4, num_decoder_layers=4, max_seq_len=seq_len
    )
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Predict and create video from frames
    predict_and_create_video(model, dataloader, device, frame_size)




C:\Users\Me\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_0.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_1.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_2.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_3.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_4.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_5.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_6.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_7.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_8.png
Saved frame: predicted_frames_transformer\batch_0_seq_0_frame_9.png
Saved frame: predicted_frames_transformer\batch_0_seq_1_frame_0.png
Saved frame: predicted_frames_transformer\batch_0_seq_1_frame_1.png
Saved frame: predicted_frames_transformer\batch_0_seq_1_frame_2.png
Saved frame: predicted_frames_transformer\batch_0_seq_1_frame_3.png
Saved frame: predicted_frames_transformer\batch_